In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np

In [3]:
from src.models.VAELightning import VAELightning

In [4]:
from dataset.BraTSDataset import BraTSDataset

In [5]:
train_path = "/cluster/51/emre/project/data/RSNA_ASNR_MICCAI_BraTS2021_TrainingData_16July2021"
data = BraTSDataset(train_path, training=True)

In [6]:
model = VAELightning(data.crop_size)

In [7]:
x,y = data[0]
x = x[None]
y = y[None]

In [9]:
y.shape

torch.Size([1, 3, 160, 192, 128])

In [8]:
model.training_step((x,y))

{'loss': tensor(1.4519, grad_fn=<AddBackward0>),
 'kl_loss': tensor(5.1257, grad_fn=<MulBackward0>),
 'reconstruct_loss': tensor(0.0478, grad_fn=<MseLossBackward0>),
 'dice_loss': tensor(0.9346, grad_fn=<RsubBackward1>)}

In [19]:
model.validation_step((x,y))

{'dice_coeff': tensor([0.0314, 0.0300, 0.0096], grad_fn=<MeanBackward1>),
 'hausdorff': tensor([[103.6774,  97.7763, 101.0247]], dtype=torch.float64)}

In [11]:
from torch.utils.data import DataLoader, random_split
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

In [12]:
train, val, test = random_split(data, [10, 5, len(data)-15])

In [13]:
train_loader = DataLoader(train, batch_size=1, num_workers=8)
val_loader = DataLoader(val, batch_size=1, num_workers=8)

In [16]:
logger = TensorBoardLogger('runs/VAE/Overfitting/logs', name='overfitting')
model = VAELightning(data.crop_size)

checkpoint_best = ModelCheckpoint(dirpath='runs/VAE/Overfitting/best_models/',
    filename='{name}_{epoch}_{val_avg_overall_dice:.2f}',
    save_top_k=5,
    monitor='val_avg_overall_dice',
    mode='max')

checkpoint_last = ModelCheckpoint(
    dirpath='runs/VAE/Overfitting/last_models/',
    filename='{name}_{epoch}',
    save_top_k=5,
    save_last=True,
    monitor="step")


trainer = Trainer(accelerator="gpu", devices=1, max_epochs=300, check_val_every_n_epoch=1,
                  callbacks=[checkpoint_best, checkpoint_last], logger=logger,
)

/rhome/emre/.cache/pypoetry/virtualenvs/ml3d-project-vk3ub45l-py3.8/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:166: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /rhome/emre/.cache/pypoetry/virtualenvs/ml3d-project ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [17]:
model = model.cuda()

In [ ]:
# start training
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: runs/VAE/Overfitting/logs/overfitting
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type               | Params
---------------------------------------------------
0 | encoder     | Encoder            | 17.6 M
1 | decoder     | Decoder            | 1.2 M 
2 | vae_decoder | VariationalDecoder | 7.2 M 
3 | vae_loss    | VAELoss            | 0     
4 | dice_loss   | DiceLoss           | 0     
---------------------------------------------------
26.0 M    Trainable params
0         Non-trainable params
26.0 M    Total params
104.151   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/rhome/emre/.cache/pypoetry/virtualenvs/ml3d-project-vk3ub45l-py3.8/lib/python3.8/site-packages/monai/metrics/hausdorff_distance.py:168: UserWarning: the ground truth of class 2 is all 0, this may result in nan/inf distance.
  warnings.warn(f"the ground truth of class {c} is all 0, this may result in nan/inf distance.")
/rhome/emre/.cache/pypoetry/virtualenvs/ml3d-project-vk3ub45l-py3.8/lib/python3.8/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/rhome/emre/.cache/pypoetry/virtualenvs/ml3d-project-vk3ub45l-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1600: PossibleUserWarning: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]